## Check HW








In [1]:
!lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         46 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  2
  On-line CPU(s) list:   0,1
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:          6
    Model:               79
    Thread(s) per core:  2
    Core(s) per socket:  1
    Socket(s):           1
    Stepping:            0
    BogoMIPS:            4399.99
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clf
                         lush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_
                         good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fm
                         a cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hyp
                         ervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd i

In [2]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            12Gi       560Mi       9.8Gi       1.0Mi       2.4Gi        11Gi
Swap:             0B          0B          0B


In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [4]:
!nvidia-smi

Mon Oct 23 17:52:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install the environment

In [5]:
#!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
!rm -rf master.zip* nvcc4jupyter-master*
!wget https://github.com/andreinechaev/nvcc4jupyter/archive/refs/heads/master.zip
!unzip master.zip
!cd nvcc4jupyter-master; ls; pip install .

--2023-10-23 17:52:05--  https://github.com/andreinechaev/nvcc4jupyter/archive/refs/heads/master.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/andreinechaev/nvcc4jupyter/zip/refs/heads/master [following]
--2023-10-23 17:52:05--  https://codeload.github.com/andreinechaev/nvcc4jupyter/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 192.30.255.121
Connecting to codeload.github.com (codeload.github.com)|192.30.255.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5206 (5.1K) [application/zip]
Saving to: ‘master.zip’

master.zip          100%[===================>]   5.08K  --.-KB/s    in 0s      

2023-10-23 17:52:06 (75.4 MB/s) - ‘master.zip’ saved [5206/5206]

Archive:  master.zip
0a71d56e5dce3ff1f0dd2c47c29367629262f527
   creating: nvcc4jupyter-master/
 extracting

In [ ]:
%load_ext nvcc_plugin

## Run a program (using Google overpowered GPUs)



### Simple mode, everything on the same file

This is the only part of the notebook that really uses the extension nvcc_plugin

In [ ]:
%%cu

#include <stdio.h>
#include <cuda.h>
#include <cuda_runtime.h>

__global__ void mykernel(void) {

}

int main(void) {
		mykernel<<<1,1>>>();
		printf("Hello world\n");
		return 0;
}

In [ ]:
%%cu
#include <cuda.h>
#include <cuda_runtime.h>
#include <stdio.h>

__global__ void add(int *a, int *b, int *c) {
    *c = *a + *b;
}

int main(void) {
    int a, b, c;				    // host copies of a, b, c
	  int *d_a, *d_b, *d_c;		// device copies of a, b, c
	  int size = sizeof(int);

	  // Allocate space for device copies of a, b, c
	  cudaMalloc((void **)&d_a, size);
	  cudaMalloc((void **)&d_b, size);
	  cudaMalloc((void **)&d_c, size);

	  // Setup input values
	  a = 2;
	  b = 7;

	  // Copy inputs to device
	  cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
	  cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);

	  // Launch add() kernel on GPU
	  add<<<1,1>>>(d_a, d_b, d_c);

	  // Copy result back to host
	  cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);

	  // Show result
	  printf("%d + %d = %d", a, b, c);

	  // Cleanup
	  cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);
	  return 0;
}

### Running code in multiple source files

In [ ]:
!mkdir cuda
!rm -rf cuda/*

In [ ]:
%%writefile cuda/kernel.cu
#include <cuda.h>

__global__ void add(int *a, int *b, int *c) {
    *c = *a + *b;
}

In [ ]:
%%writefile cuda/main.cu
#include <cuda.h>
#include <cuda_runtime.h>
#include <stdio.h>

__global__ void add(int *a, int *b, int *c);

int main(void) {
	int a, b, c;				    // host copies of a, b, c
	int *d_a, *d_b, *d_c;		// device copies of a, b, c
	int size = sizeof(int);

	// Allocate space for device copies of a, b, c
	cudaMalloc((void **)&d_a, size);
	cudaMalloc((void **)&d_b, size);
	cudaMalloc((void **)&d_c, size);

	// Setup input values
	a = 2;
	b = 7;

	// Copy inputs to device
	cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
	cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);

	// Launch add() kernel on GPU
	add<<<1,1>>>(d_a, d_b, d_c);

	// Copy result back to host
	cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);

	// Show result
	printf("%d + %d = %d", a, b, c);

	// Cleanup
	cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);
	return 0;
}

In [ ]:
!nvcc cuda/*.cu -o cuda/program
!./cuda/program
!rm -rf ./cuda/program

## Mount your Google Drive

In [ ]:
# Mount Google Drive
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/gdrive')

In [ ]:
# Copy source code to your folder
!cp /content/gdrive/MyDrive/CAP_practica2/* cuda/
# List folder
!ls -la cuda/

# Ejercicio 1

In [ ]:
!mkdir cuda/Ejercicio1

In [ ]:
%%writefile cuda/Ejercicio1/stencil1d.c
#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>

void stencil1D(int *in, int *out, int size) {
    for (int i = 1; i < size - 1; i++) {
        // Calcula el valor en out[i] como el promedio de los valores en in[i-1], in[i] y in[i+1]
        out[i] = (in[i - 1] + in[i] + in[i + 1]) / 3;
    }

    // Trata los casos especiales de los bordes izquierdo y derecho
    out[0] = (in[0] + in[1]) / 2;
    out[size - 1] = (in[size - 2] + in[size - 1]) / 2;
}

int main(int argc, char **argv) {
    //int N = 100000;

    if(argc != 2){
        printf("Introduce el tamaño del array como argumento.");
    }
    int N = atoi(argv[1]);

    // Reservamos memoria para los arrays
    int *input = (int *)malloc(N * sizeof(int));
    int *output = (int *)malloc(N * sizeof(int));

    // inicializamos el array a valores aleatorios
    for (int i = 0; i < N; i++) {
        input[i] = rand() % 100;
    }

    // empezamos a medir el tiempo
    struct timeval start, end;
    gettimeofday(&start, NULL);

    // llamada a stencil1d
    stencil1D(input, output, N);

    gettimeofday(&end, NULL);


	  printf("%0.8f\n", (end.tv_sec - start.tv_sec) + 1e-6*(end.tv_usec - start.tv_usec));

    // liberamos memoria
    free(input);
    free(output);

    return 0;
}


In [ ]:
%%writefile cuda/Ejercicio1/stencil1dgpu.cu
#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>


#define STENCIL_RADIUS 3

__global__ void stencil1D(int *in, int *out, int size, int stencil_radius) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // memoria compartida
    extern __shared__ int shared_data[];


    int shared_idx = threadIdx.x + stencil_radius;
    if (idx < size) {
        shared_data[shared_idx] = in[idx];
    }

    // Cargamos los datos de los bordes en memoria compartida
    if (threadIdx.x < stencil_radius) {
        int left = idx - stencil_radius;
        int right = idx + blockDim.x;
        if (left >= 0) {
            shared_data[shared_idx - stencil_radius] = in[left];
        }
        if (right < size) {
            shared_data[shared_idx + blockDim.x] = in[right];
        }
    }

    __syncthreads();

    if (idx < size) {
        int result = 0;
        for (int i = -stencil_radius; i <= stencil_radius; i++) {
            result += shared_data[shared_idx + i];
        }
        out[idx] = result / (2 * stencil_radius + 1);
    }
}


int main(int argc, char **argv) {
    //int N = 100000;
    int block_size = 256;

    if(argc != 2){
        printf("Introduce el tamaño del array como argumento.");
    }
    int N = atoi(argv[1]);

    int *input = (int *)malloc(N * sizeof(int));
    int *output = (int *)malloc(N * sizeof(int));

    // Inicializamos el array con valores aleatorios
    for (int i = 0; i < N; i++) {
        input[i] = rand() % 100;
    }

    // Reserva de memoria para la GPU
    int *d_input, *d_output;
    cudaMalloc((void **)&d_input, N * sizeof(int));
    cudaMalloc((void **)&d_output, N * sizeof(int));

    cudaMemcpy(d_input, input, N * sizeof(int), cudaMemcpyHostToDevice);

    int num_blocks = (N + block_size - 1) / block_size;


    struct timeval start, end;
    gettimeofday(&start, NULL);

    // Lanzamos el kernel de GPU
    stencil1D<<<num_blocks, block_size, (block_size + 2 * STENCIL_RADIUS) * sizeof(int)>>>(d_input, d_output, N, STENCIL_RADIUS);

    cudaDeviceSynchronize();

    gettimeofday(&end, NULL);

	  printf("%0.8f\n", (end.tv_sec - start.tv_sec) + 1e-6*(end.tv_usec - start.tv_usec));

    cudaMemcpy(output, d_output, N * sizeof(int), cudaMemcpyDeviceToHost);

    // Liberamos memoria
    cudaFree(d_input);
    cudaFree(d_output);
    free(input);
    free(output);

    return 0;
}


In [ ]:
!cd cuda/Ejercicio1 && gcc -Wall -D_GNU_SOURCE -std=gnu99  -o  cpu stencil1d.c
!nvcc cuda/Ejercicio1/stencil1dgpu.cu -o cuda/Ejercicio1/gpu


In [ ]:
%%writefile cuda/Ejercicio1/times.sh

start=100000
end=1000000
step=100000

for N in $(seq $start $step $end); do
    TimesEj1Cpu=$(./cuda/Ejercicio1/cpu $N)
    TimesEj1Gpu=$(./cuda/Ejercicio1/gpu $N)
    echo "$TimesEj1Cpu $TimesEj1Gpu"
done


In [ ]:
!chmod +x ./cuda/Ejercicio1/times.sh
!./cuda/Ejercicio1/times.sh

# Ejercicio 2

## GreyScale P1


In [ ]:
!mkdir cuda/Ejercicio2
# Create soft links
!ln -s /content/gdrive/MyDrive/CAP_practica2/images cuda/Ejercicio2

In [ ]:
%%writefile cuda/Ejercicio2/greyScale.c
#include <stdio.h>
#include <stdint.h>
#include <math.h>
#include <sys/time.h>
#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

static inline void getRGB(uint8_t *im, int width, int height, int nchannels, int x, int y, int *r, int *g, int *b)
{

    unsigned char *offset = im + (x + width * y) * nchannels;
    *r = offset[0];
    *g = offset[1];
    *b = offset[2];
}

int main(int nargs, char **argv)
{
    int width, height, nchannels;
    struct timeval fin,ini;

    if (nargs < 2)
    {
        printf("Usage: %s <image1> [<image2> ...]\n", argv[0]);
    }
    // For each image
    // Bucle 0
    for (int file_i = 1; file_i < nargs; file_i++)
    {
        printf("[info] Processing %s\n", argv[file_i]);
        /****** Reading file ******/
        uint8_t *rgb_image = stbi_load(argv[file_i], &width, &height, &nchannels, 4);
        if (!rgb_image)
        {
            perror("Image could not be opened");
        }

        /****** Allocating memory ******/
        // - RGB2Grey
        uint8_t *grey_image = malloc(width * height);
        if (!grey_image)
        {
            perror("Could not allocate memory");
        }

        // - Filenames
        for (int i = strlen(argv[file_i]) - 1; i >= 0; i--)
        {
            if (argv[file_i][i] == '.')
            {
                argv[file_i][i] = 0;
                break;
            }
        }

        char *grey_image_filename = 0;
        asprintf(&grey_image_filename, "%s_grey.jpg", argv[file_i]);
        if (!grey_image_filename)
        {
            perror("Could not allocate memory");
            exit(-1);
        }

        /****** Computations ******/
        printf("[info] %s: width=%d, height=%d, nchannels=%d\n", argv[file_i], width, height, nchannels);

        if (nchannels != 3 && nchannels != 4)
        {
            printf("[error] Num of channels=%d not supported. Only three (RGB), four (RGBA) are supported.\n", nchannels);
            continue;
        }

        gettimeofday(&ini,NULL);
        // RGB to grey scale
        int r, g, b;
        for (int j = 0; j < height; j++)
        {
            for (int i = 0; i < width; i++)
            {
                getRGB(rgb_image, width, height, 4, i, j, &r, &g, &b);
                grey_image[j * width + i] = (int)(0.2989 * r + 0.5870 * g + 0.1140 * b);
            }
        }

        stbi_write_jpg(grey_image_filename, width, height, 1, grey_image, 10);
        free(rgb_image);

        gettimeofday(&fin,NULL);

	    printf("Tiempo: %f\n", ((fin.tv_sec*1000000+fin.tv_usec)-(ini.tv_sec*1000000+ini.tv_usec))*1.0/1000000.0);
        free(grey_image_filename);
    }
}


In [ ]:
!cd cuda/Ejercicio2 && gcc -Wall -D_GNU_SOURCE -std=gnu99  -o greyScale greyScale.c -lm
!mkdir cuda/Ejercicio2/images
!cd cuda/Ejercicio2/ && ./greyScale ./images/HD.jpg
!cd cuda/Ejercicio2/ && ./greyScale ./images/SD.jpg
!cd cuda/Ejercicio2/ && ./greyScale ./images/FHD.jpg
!cd cuda/Ejercicio2/ && ./greyScale ./images/4k.jpg
!cd cuda/Ejercicio2/ && ./greyScale ./images/8k.jpg

## GreyScale P2

In [ ]:
%%writefile cuda/Ejercicio2/greyScaleGpu.cu
#include <stdio.h>
#include <stdint.h>
#include <math.h>
#include <sys/time.h>
#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"


#define BLOCK_SIZE 256

__global__ void multiplyRatios(uint8_t *in, float *out, float* d_in_mul) {
    __shared__ uint8_t aux[BLOCK_SIZE];
    int gindex = threadIdx.x + blockIdx.x * blockDim.x;
    int lindex = threadIdx.x;

    aux[lindex] = in[gindex] * d_in_mul[lindex];

    __syncthreads();

    out[gindex] = aux[lindex];
}

__global__ void rgbToGrey(float *in, uint8_t *out) {
  int gindex = threadIdx.x + blockIdx.x * blockDim.x;
  int gindex_temp = gindex * 4;
  int result = 0;
  for (int offset = 0 ; offset < 4 ; offset++) {
    result += in[gindex_temp + offset];
  }

  out[gindex] = result;
}

int main(int nargs, char **argv)
{
    int width, height, nchannels;
    struct timeval fin,ini;
    uint8_t *d_in, *d_out;
    float *d_out_mul, *d_in_mul;
    float mul_ratios[BLOCK_SIZE];

    if (nargs < 2)
    {
        printf("Usage: %s <image1> [<image2> ...]\n", argv[0]);
    }
    // For each image
    // Bucle 0
    for (int file_i = 1; file_i < nargs; file_i++)
    {
        printf("[info] Processing %s\n", argv[file_i]);
        /****** Reading file ******/
        uint8_t *rgb_image = stbi_load(argv[file_i], &width, &height, &nchannels, 4);
        if (!rgb_image)
        {
            perror("Image could not be opened");
        }

        /****** Allocating memory ******/
        // - RGB2Grey
        uint8_t *grey_image = (uint8_t *) calloc(width, height);
        if (!grey_image)
        {
            perror("Could not allocate memory");
        }

        int size = width * height;

        cudaMalloc((void **) &d_in, size * 4 * sizeof(uint8_t));
        cudaMalloc((void **) &d_out, size * sizeof(uint8_t));
        cudaMalloc((void **) &d_out_mul, size * 4 * sizeof(float));
        cudaMalloc((void **) &d_in_mul, BLOCK_SIZE * sizeof(float));
        cudaMemcpy(d_in, rgb_image, size * 4 * sizeof(uint8_t), cudaMemcpyHostToDevice);
        cudaMemcpy(d_out, grey_image, size * sizeof(uint8_t), cudaMemcpyHostToDevice);

        //inicializamos el array de mul_ratios
        for (int i = 0; i < BLOCK_SIZE; i += 4) {
          mul_ratios[i] = 0.2989;
          mul_ratios[i + 1] = 0.5870;
          mul_ratios[i + 2] = 0.1140;
          mul_ratios[i + 3] = 0;
        }

        // - Filenames
        for (int i = strlen(argv[file_i]) - 1; i >= 0; i--)
        {
            if (argv[file_i][i] == '.')
            {
                argv[file_i][i] = 0;
                break;
            }
        }

        char *grey_image_filename = 0;
        asprintf(&grey_image_filename, "%s_grey.jpg", argv[file_i]);
        if (!grey_image_filename)
        {
            perror("Could not allocate memory");
            exit(-1);
        }

        /****** Computations ******/
        printf("[info] %s: width=%d, height=%d, nchannels=%d\n", argv[file_i], width, height, nchannels);

        if (nchannels != 3 && nchannels != 4)
        {
            printf("[error] Num of channels=%d not supported. Only three (RGB), four (RGBA) are supported.\n", nchannels);
            continue;
        }

        gettimeofday(&ini,NULL);

        cudaMemcpy(d_in_mul, mul_ratios, sizeof(mul_ratios), cudaMemcpyHostToDevice);
        multiplyRatios<<<(size*4)/BLOCK_SIZE, BLOCK_SIZE>>>(d_in, d_out_mul, d_in_mul);
        rgbToGrey<<<size/BLOCK_SIZE, BLOCK_SIZE>>>(d_out_mul, d_out);
        cudaMemcpy(grey_image, d_out, size * sizeof(uint8_t), cudaMemcpyDeviceToHost);

        stbi_write_jpg(grey_image_filename, width, height, 1, grey_image, 10);
        free(rgb_image);
        gettimeofday(&fin,NULL);

	      printf("Tiempo: %f\n", ((fin.tv_sec*1000000+fin.tv_usec)-(ini.tv_sec*1000000+ini.tv_usec))*1.0/1000000.0);
        free(grey_image_filename);
        cudaFree(d_in);
        cudaFree(d_out);
        cudaFree(d_out_mul);
    }
}


In [ ]:
!cd cuda/Ejercicio2/images && rm *_grey.jpg
!cd cuda/Ejercicio2 && nvcc -o greyScaleGpu *.cu -lm
!cd cuda/Ejercicio2/ && ./greyScaleGpu ./images/HD.jpg
!cd cuda/Ejercicio2/ && ./greyScaleGpu ./images/SD.jpg
!cd cuda/Ejercicio2/ && ./greyScaleGpu ./images/FHD.jpg
!cd cuda/Ejercicio2/ && ./greyScaleGpu ./images/4k.jpg
!cd cuda/Ejercicio2/ && ./greyScaleGpu ./images/8k.jpg